In [98]:
from math import log, pow, exp, sqrt, fabs, e
from scipy.stats import norm
import pandas as pd
from datetime import date, datetime
import numpy as np
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta
import matplotlib.pyplot as plt
import os

In [99]:
work_dir = os.getcwd()

In [17]:
class Option:
    """
    Calculate the Greeks letter for Option 
    """
    def __init__(self, right, s, k, eval_date, exp_date, price = None, rf = 0.0, vol = 0.3,
                 div = 0):
        self.k = float(k)
        self.s = float(s)
        self.rf = float(rf)
        self.vol = float(vol)
        self.eval_date = eval_date
        self.exp_date = exp_date
        self.t = self.calculate_t()
        self.price = price
        self.right = right   ## 'Call' or 'Put'
        self.div = div
        self.d1 = (log(self.s/self.k) + (self.rf + self.div + pow(self.vol, 2)/2 ) * self.t) / (self.vol * sqrt(self.t))
        self.d2 = self.d1 - self.vol * sqrt(self.t)
        self.greeks = []

    def calculate_t(self):
        if isinstance(self.eval_date, str):
            if '/' in self.eval_date:
                (day, month, year) = self.eval_date.split('/')
            else:
                (day, month, year) = self.eval_date[6:8], self.eval_date[4:6], self.eval_date[0:4]
            d0 = date(int(year), int(month), int(day))
        elif type(self.eval_date)==float or type(self.eval_date)==long or type(self.eval_date)==np.float64:
            (day, month, year) = (str(self.eval_date)[6:8], str(self.eval_date)[4:6], str(self.eval_date)[0:4])
            d0 = date(int(year), int(month), int(day))
        else:
            d0 = self.eval_date 

        if isinstance(self.exp_date, str):
            if '/' in self.exp_date:
                (day, month, year) = self.exp_date.split('/')
            else:
                (day, month, year) = self.exp_date[6:8], self.exp_date[4:6], self.exp_date[0:4]
            d1 = date(int(year), int(month), int(day))
        elif type(self.exp_date)==float or type(self.exp_date)==long or type(self.exp_date)==np.float64:
            (day, month, year) = (str(self.exp_date)[6:8], str(self.exp_date)[4:6], str(self.exp_date)[0:4])
            d1 = date(int(year), int(month), int(day))
        else:
            d1 = self.exp_date

        t = np.busday_count(d0, d1)/252
        if t < 0:
            print("The input dates error!")
        return t
    
    def get_price(self):
        d1 = self.d1
        d2 = self.d2
        if self.right == 'Call':
            self.calc_price = ( norm.cdf(d1) * self.s * exp(-self.div*self.t) - norm.cdf(d2) * self.k * exp( -self.rf * self.t ) )
        elif self.right == 'Put':
            self.calc_price =  ( -norm.cdf(-d1) * self.s * exp(-self.div*self.t) + norm.cdf(-d2) * self.k * exp( -self.rf * self.t ))
        return self.calc_price
        
    def get_delta(self):
        if self.right == 'Call':
            self.delta = norm.cdf(self.d1)
        elif self.right == 'Put':
            self.delta = -norm.cdf(-self.d1) 
 
 
    def get_theta(self):
        
        prob_density = norm.pdf(self.d1)
        if self.right == 'Call':
            self.theta = (-self.vol * self.s * prob_density) / (2 * np.sqrt(self.t)) - self.rf * self.k * np.exp(-self.rf * self.t) * norm.cdf(self.d2)
        elif self.right == 'Put':    
            self.theta = (-self.vol * self.s * prob_density) / (2 * np.sqrt(self.t)) + self.rf * self.k * np.exp(-self.rf * self.t) * norm.cdf(self.d2)
        self.theta = self.theta/365
        
    def get_gamma(self):
        self.gamma = e ** (-self.div * self.t) * norm.pdf(self.d1) / (self.s * self.vol* self.t ** 0.5)
        
 
    def get_vega(self):
        prob_density = norm.pdf(self.d1)
        self.vega = 0.01 * self.s * prob_density * np.sqrt(self.t)
        
    def get_greeks(self):
        self.get_delta()
        self.get_theta()
        self.get_gamma()
        self.get_vega()
        self.greeks = [self.delta, self.gamma, self.theta, self.vega]
        return self.greeks
    
    def describe_option(self):
        if len(self.greeks) < 1:
            greeks = self.get_greeks()
        else:
            print(self.right + " Delta: " + str(self.delta)) 
            print(self.right + " Gamma: " + str(self.gamma)) 
            print(self.right + " Theta: " + str(self.theta))   
            print(self.right + " Vega: " + str(self.vega))
        

In [2]:
class Options_strategy:
    """
    Calculate the Greeks for the Option Strategies
    """
    def __init__(self, df_options): 
        self.df_options = df_options     
        self.greeks = []
        self.s = int(df_options['現貨價格'][0])
        self.s_list = self.moneyness_list(gapType = 'month', gapNum = 10)
        self.position = ""
        self.payoff = []
        self.bpPoints = []
        
    def get_greeks(self):
        self.delta = 0
        self.gamma = 0
        self.theta = 0
        self.vega = 0
        
        for k, v in self.df_options.iterrows():
 
              
            opt = Option(s = v['現貨價格'], k=v['履約價'], eval_date = v['今日'], 
                         exp_date=v['到期日'], rf = 0.0, vol = v['IV'],
                         right = v['買賣權'])

            greeks = opt.get_greeks()

            self.delta += float(v['部位']) * greeks[0]
            self.gamma += float(v['部位']) * greeks[1]
            self.theta += float(v['部位']) * greeks[2]
            self.vega += float(v['部位']) * greeks[3]

        self.greeks = [self.delta, self.gamma, self.theta, self.vega]
        return self.greeks
    
    
    def moneyness_list(self, gapType, gapNum):
        # Find out the list of OTM/ITM stock prices
        if gapType == "month":
            gap = 100
        elif gapType == "week":
            gap = 50
        else:
            print("Option Type error")
        
        S_list = []
        S_floor = int(self.s - self.s%gap)
        S_ceil = int(S_floor + gap)
        S_list.append(S_floor)
        S_list.append(S_ceil)

        for i in range(1, gapNum):
            S_list.append(int(S_floor - gap*i))
            S_list.append(int(S_ceil + gap*i))
        S_list.sort()

        return S_list
    
    
    def get_slist(self):
        return self.s_list
    
    def get_payoff(self):
        
        self.payoff = [0, ]*len(self.s_list)
        
        for k, v in self.df_options.iterrows():
            #record the text position
            if v['部位'][0] == '-':
                temptxt = "Sell " + str(v['部位'][1:]) + " " + str(v['買賣權']) + "@" + str(v['履約價']) 
            else:
                temptxt = "Buy " + str(v['部位']) + " " + str(v['買賣權']) + "@" + str(v['履約價'])
            self.position += temptxt + "\n"
            # calculate the pnl
            if v['買賣權'] == 'Call':
                for i, s in enumerate(self.s_list):
                    price = float(v['結算價'])
                    k = float(v['履約價'])
                    pnl = float(v['部位']) * max(-price, s-k-price)
                    self.payoff[i] += pnl

                    
            elif v['買賣權'] == 'Put':
                for i, s in enumerate(self.s_list):
                    price = float(v['結算價'])
                    k = float(v['履約價'])
                    pnl = float(v['部位']) * max(-price, k-s-price)
                    self.payoff[i] += pnl
            else:
                print("Option Type Error")

                
        return self.payoff
    
    
    
    def get_bpPoints(self):
        
        if self.payoff == []:
            self.get_payoff()
            
        try:
            tmpP = self.payoff[0]
            tmpS = self.s_list[0]
            for i in range(1, len(self.payoff)):
                if self.payoff[i]*tmpP > 0:
                    tmpP = self.payoff[i]
                    tmpS = self.s_list[i]
                else:
                    bp = (-tmpP/(self.payoff[i] - tmpP))*(self.s_list[i] - tmpS) + tmpS
                    self.bpPoints.append(int(bp))
                    tmpP = self.payoff[i]
                    tmpS = self.s_list[i]
                                            
        except:
            print("No value in Payoff")
            raise
            
    def get_maxPayoff(self):
        if self.payoff == []:
            self.get_payoff()
            
        if self.payoff.count(max(self.payoff)) < 2:
            return float("inf")
        else:
            return round(max(self.payoff), 2)
    
    def get_maxLoss(self):
        if self.payoff == []:
            self.get_payoff()
        
        if self.payoff.count(min(self.payoff)) < 2:
            return float("-inf")
        else:
            return round(min(self.payoff), 2)
    
        
    def portfolio_payoff(self, fileName):
        
        if self.payoff == []:
            self.get_payoff()
            
        self.get_bpPoints()
        
        
        plt.style.use('ggplot')
        plt.figure(figsize=(10,5))
        plt.plot(self.s_list[:-1], self.payoff[:-1],  marker='o') #make the plot symmetry
        plt.axhline(y = 0, linestyle = 'dashed', color = 'black')
        
        bptxt = ""
        for bp in self.bpPoints:
            plt.plot(bp, 0, marker = "*", color = 'Blue', markersize=11)
            plt.text(bp, 0, str(bp))
            bptxt += str(bp)+" "
            
        textstr = '\n'.join((
                r'maxPayoff = %.2f' % (self.get_maxPayoff(), ),
                r'bpPoints = %s' % (bptxt, ), 
                r'maxLoss= %.2f' % (self.get_maxLoss(), )))
        
        plt.text(min(self.s_list), max(self.payoff), textstr, fontsize=10,
        verticalalignment='top', bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'))
        
        
        title_name = "PayOff: " + self.position
        plt.title(title_name, fontsize=12)
        plt.xlabel('Stock Price', fontsize=11)
        plt.ylabel('PnL', fontsize=11) 
        plt.savefig(os.path.join(work_dir, 'Graph', fileName))
        plt.show()
        
    
    def describe_portfolio(self):
        if len(self.greeks) < 1:
            greeks = self.get_greeks()
        else:
            print("Aggr. Delta: " + str(self.delta)) 
            print("Aggr. Gamma: " + str(self.gamma)) 
            print("Aggr. Theta: " + str(self.theta))   
            print("Aggr. Vega: " + str(self.vega))
        